In [7]:
import os
import requests

In [8]:
from dotenv import load_dotenv
load_dotenv()

True

In [9]:
locations_url = "https://apidojo-booking-v1.p.rapidapi.com/locations/auto-complete"

In [12]:
querystring = {"languagecode":"de","text":"Esslingen am Neckar"}

In [143]:
booking_session = requests.Session()
booking_session.headers.update( {
    'x-rapidapi-host': "apidojo-booking-v1.p.rapidapi.com",
    'x-rapidapi-key': os.environ["RAPIDAPI_BOOKING_COM_API_KEY"]
    })

## Get destination id of the city

List by map requires this

In [14]:
r = booking_session.get(locations_url, params=querystring)

In [21]:
dest_id = r.json()[0]["dest_id"]

In [22]:
print(dest_id)

-1767916


## Get hotels from single location

In [37]:
list_url = "https://apidojo-booking-v1.p.rapidapi.com/properties/list"

In [38]:
querystring = {"price_filter_currencycode":"USD",
               "travel_purpose":"leisure",
               "categories_filter":"price::0-60,free_cancellation::1,class::1,class::0,class::2",
               "search_id":"none",
               "order_by":"price",
               "children_qty":"0",
               "languagecode":"de",
               "children_age":"",
               "search_type":"city",
               "offset":"0",
               "dest_ids":dest_id,
               "guest_qty":"1",
               "arrival_date":"2019-09-13",
               "departure_date":"2019-09-14",
               "room_qty":"1"}

In [39]:
r = booking_session.get(list_url, params=querystring)

In [43]:
r.json()

{'search_radius': 25.0,
 'page_loading_threshold': 10,
 'room_distribution': [{'adults': '1', 'children': []}],
 'extended_count': 165,
 'ranking_version': 5,
 'result': [{'breakfast_review_score': {'review_count': 39,
    'rating': 7.6,
    'review_score_word': 'Gut'},
   'block_ids': ['25029101_200123460_1_2_0'],
   'district_id': 0,
   'min_total_price': 68,
   'hotel_name_trans': 'Hotel-Restaurant Kelter',
   'class': 2.0,
   'currencycode': 'EUR',
   'cc_required': 1,
   'mobile_discount_percentage': 0,
   'ufi': -1767916,
   'district': '',
   'zip': '73733',
   'location_score_reviews_nr': 165,
   'main_photo_id': 54237181,
   'city_trans': 'Esslingen',
   'urgency_c': [0],
   'checkout': {'until': '10:30', 'from': '07:00'},
   'districts': '',
   'cleanliness_score': 8.3,
   'hotel_include_breakfast': 0,
   'longitude': 9.28754232823849,
   'bwallet': {'hotel_eligibility': 1},
   'distance': '1.84',
   'address': 'Kelterstr. 104',
   'available_rooms': 2,
   'hotel_facilities':

In [41]:
r.json()["result"][0]

{'breakfast_review_score': {'review_count': 39,
  'rating': 7.6,
  'review_score_word': 'Gut'},
 'block_ids': ['25029101_200123460_1_2_0'],
 'district_id': 0,
 'min_total_price': 68,
 'hotel_name_trans': 'Hotel-Restaurant Kelter',
 'class': 2.0,
 'currencycode': 'EUR',
 'cc_required': 1,
 'mobile_discount_percentage': 0,
 'ufi': -1767916,
 'district': '',
 'zip': '73733',
 'location_score_reviews_nr': 165,
 'main_photo_id': 54237181,
 'city_trans': 'Esslingen',
 'urgency_c': [0],
 'checkout': {'until': '10:30', 'from': '07:00'},
 'districts': '',
 'cleanliness_score': 8.3,
 'hotel_include_breakfast': 0,
 'longitude': 9.28754232823849,
 'bwallet': {'hotel_eligibility': 1},
 'distance': '1.84',
 'address': 'Kelterstr. 104',
 'available_rooms': 2,
 'hotel_facilities': '2,70,8,4,305,47,108,80,16,3,76,107,96,160,46',
 'price_is_final': 1,
 'accommodation_type_name': 'Pensionen',
 'is_city_center': 0,
 'urgency_room_c': 1,
 'selected_review_topic': None,
 'is_genius_deal': 0,
 'timezone': 'E

## Get hotels from a bounding box with edge size $a$

In [117]:
a = 20000

In [118]:
loc = (48.7361, 9.2977)

In [119]:
import shapely
from pyproj import Transformer

In [120]:
epsg_gps = 4326
epsg_utm32 = 32632

In [121]:
transformer_gps_to_utm = Transformer.from_crs(epsg_gps, epsg_utm32)
transformer_utm_to_gps = Transformer.from_crs(epsg_utm32, epsg_gps)

In [122]:
loc_x, loc_y = transformer_gps_to_utm.transform(*loc)

In [146]:
loc_ne_x = loc_x + a/2
loc_ne_y = loc_y + a/2

In [147]:
loc_sw_x = loc_x - a/2
loc_sw_y = loc_y - a/2 

In [148]:
loc_bb_ne = transformer_utm_to_gps.transform(loc_ne_x, loc_ne_y)

In [149]:
loc_bb_sw = transformer_utm_to_gps.transform(loc_sw_x, loc_sw_y)

In [150]:
import folium

In [151]:
m = folium.Map(location=[*loc])

In [152]:
folium.Marker([*loc_bb_ne], popup='North East').add_to(m)
folium.Marker([*loc_bb_sw], popup='South West').add_to(m)

In [154]:
m

In [155]:
map_url = "https://apidojo-booking-v1.p.rapidapi.com/properties/list-by-map"

In [156]:
bbox_string = f"{loc_bb_sw[0]},{loc_bb_ne[0]},{loc_bb_sw[1]},{loc_bb_ne[1]}"
print(bbox_string)

48.646410446861964,48.82562542005355,9.161410619476062,9.434474679411416


In [157]:
querystring = {"search_id":"none",
               "price_filter_currencycode":"EUR",
               "languagecode":"de",
               "travel_purpose":"leisure",
               "categories_filter":"price::0-60,free_cancellation::1,class::1,class::0,class::2",
               "children_qty":"0",
               "order_by":"price",
               "guest_qty":"1",
               "room_qty":"1",
               "departure_date":"2019-09-14",
               "bbox":bbox_string,
               "arrival_date":"2019-09-13"}

In [158]:
r = booking_session.get(map_url, params=querystring)

In [162]:
r.json()["result"][0]

{'breakfast_review_score': {'review_score_word': 'Ansprechend',
  'rating': 6.8,
  'review_count': 454},
 'hotel_name': 'a&o Stuttgart City',
 'default_language': 'xu',
 'preferred': 1,
 'business_review_score_word': 'Gut',
 'district_id': 808,
 'bwallet': {'hotel_eligibility': 0},
 'review_nr': 3460,
 'class_is_estimated': 1,
 'checkout': {'until': '10:00', 'from': ''},
 'mobile_discount_percentage': 0,
 'in_best_district': 0,
 'accommodation_type_name': 'Hotel',
 'is_mobile_deal': 0,
 'cleanliness_score_word': 'Gut',
 'price_is_final': 0,
 'main_photo_url': 'http://ac-r.static.booking.cn/xdata/images/hotel/square60/165054580.jpg?k=516fd1fc0c3d0316876d8b5addddcde764b40ae634825f4dd19dbeb7af0c38b1&o=',
 'price_breakdown': {'all_inclusive_price': 19.65,
  'sum_excluded_raw': '0.00',
  'currency': 'EUR',
  'has_fine_print_charges': 1,
  'has_tax_exceptions': 0,
  'has_incalculable_charges': 0,
  'gross_price': 19.65},
 'block_ids': ['142152007_205755946_1_2_0'],
 'latitude': 48.79307,
 'h